<a href="https://colab.research.google.com/github/nhombaitap123/baitaplonmonoop/blob/master/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [215]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [216]:
%cd /content/drive/'My Drive'/IMDB

/content/drive/My Drive/IMDB


In [217]:
movie_reviews = pd.read_csv("IMDB_Dataset.csv")
movie_reviews.isnull().values.any()
movie_reviews.shape

(50000, 2)

In [218]:
movie_reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [219]:
movie_reviews["review"][3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

### Data Preprocessing

In [0]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [0]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

### Store reviews in a new list

In [0]:
X = []
sentences = list(movie_reviews['review'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [223]:
X[3]

'Basically there a family where little boy Jake thinks there a zombie in his closet his parents are fighting all the time This movie is slower than soap opera and suddenly Jake decides to become Rambo and kill the zombie OK first of all when you re going to make film you must Decide if its thriller or drama As drama the movie is watchable Parents are divorcing arguing like in real life And then we have Jake with his closet which totally ruins all the film expected to see BOOGEYMAN similar movie and instead watched drama with some meaningless thriller spots out of just for the well playing parents descent dialogs As for the shots with Jake just ignore them '

### Convert our labels into digits

In [0]:
y = movie_reviews['sentiment']

y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Preparing the Embedding Layer

In [0]:
#the Tokenizer class from the keras.preprocessing.text module to create a word-to-index dictionary
tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [0]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

#We set the maximum size of each list to 100. You can try a different size. 
#The lists with size greater than 100 will be truncated to 100. For the lists that have length less than 100, 
#we will add 0 at the end of the list until it reaches the max length. This process is called padding.

# maxlen = 100
maxlen_train = max([len(s) for s in X_train])

#X_train or X_test, you will see that all the lists have same length i.e. 100
#vocabulary_size variable now contains a value 92547 which means that our corpus has 92547 unique words
X_train_1 = pad_sequences(X_train, padding='post', maxlen=maxlen_train)
X_test_1 = pad_sequences(X_test, padding='post', maxlen=maxlen_train)
X_train_2 = pad_sequences(X_train, padding='post', maxlen=100)
X_test_2 = pad_sequences(X_test, padding='post', maxlen=100)

In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [0]:
# embedding matrix where each row number will correspond to the index of the word in the corpus
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
  embedding_vector = embeddings_dictionary.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

### Model 1

In [0]:
#Text Classification with Simple Neural Network
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
model = Sequential()

embedding_layer = Embedding(vocab_size, 100, input_length=maxlen_train, trainable=False)
model.add(embedding_layer)

In [0]:
#simple convolutional neural network with 1 convolutional layer and 1 pooling layer
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

from keras import backend as K

def recall_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

In [232]:
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 2004, 100)         9254700   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 2000, 128)         64128     
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 11        
Total params: 9,320,129
Trainable params: 65,429
Non-trainable params: 9,254,700
_________________________________________________________________
None


In [0]:
#train our model and evaluate it on the training set
history = model.fit(X_train_1, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

loss, accuracy, f1_score, precision, recall = model.evaluate(X_test_1, y_test, verbose=1)


Train on 32000 samples, validate on 8000 samples
Epoch 1/6
32000/32000 [==============================] - 466s 15ms/step - loss: 0.6327 - acc: 0.6647 - f1_m: 0.6185 - precision_m: 0.6510 - recall_m: 0.6473 - val_loss: 0.5314 - val_acc: 0.7539 - val_f1_m: 0.7643 - val_precision_m: 0.7305 - val_recall_m: 0.8040
Epoch 2/6
32000/32000 [==============================] - 465s 15ms/step - loss: 0.4640 - acc: 0.7859 - f1_m: 0.7899 - precision_m: 0.7728 - recall_m: 0.8147 - val_loss: 0.4225 - val_acc: 0.8165 - val_f1_m: 0.8152 - val_precision_m: 0.8164 - val_recall_m: 0.8159
Epoch 3/6
19840/32000 [=================>............] - ETA: 2:42 - loss: 0.3715 - acc: 0.8427 - f1_m: 0.8432 - precision_m: 0.8310 - recall_m: 0.8603

In [0]:
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test precision", precision)
print("Test Recall", recall)
print("Test f1-score", f1_score)

In [0]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

### Model 2

In [0]:
#Text Classification with Simple Neural Network
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

model2 = Sequential()

#The embedding layer will have an input length of 100, the output vector dimension will also be 100. 
#The vocabulary size will be 92547 words.
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100 , trainable=False)
model2.add(embedding_layer)

In [0]:
#simple convolutional neural network with 1 convolutional layer and 1 pooling layer
model2.add(Conv1D(128, 5, activation='relu'))
model2.add(GlobalMaxPooling1D())
model2.add(Dense(1, activation='sigmoid'))

from keras import backend as K

def recall_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

In [0]:
print(model2.summary())

In [0]:
#train our model and evaluate it on the training set
history2 = model2.fit(X_train_2, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

loss, accuracy, f1_score, precision, recall = model2.evaluate(X_test_2, y_test, verbose=1)


In [0]:
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test precision", precision)
print("Test Recall", recall)
print("Test f1-score", f1_score)

In [0]:
import matplotlib.pyplot as plt

plt.plot(history2.history['acc'])
plt.plot(history2.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

### Prediction

In [0]:
instance = X[41244]
print(instance)

In [0]:
instance = tokenizer.texts_to_sequences(instance)

#pad our input sequence as we did for our corpus
flat_list = []
for sublist in instance:
    for item in sublist:
        flat_list.append(item)

flat_list = [flat_list]

instance_1 = pad_sequences(flat_list, padding='post', maxlen=maxlen_train)
instance_2 = pad_sequences(flat_list, padding='post', maxlen=100)

In [0]:
model.predict(instance_1)

In [0]:
model2.predict(instance_2)